<a href="https://colab.research.google.com/github/JaiderSantos/metodos1_JaiderSantos_NicolasGomez/blob/main/Parcial2/Parcial_2_Algebra_Lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import sympy as sym
import numpy as np
from scipy import integrate
import math as math
import matplotlib.pyplot as plt
from scipy.special import roots_legendre
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
import copy

C = np.array([8.51, 10.68, 12.24, 13.66, 15.37, 17.15, 19.66, 24.69])
Ca = 1000000


def func_de_retorno(C, w, Ca):
    # Ensure w has the same length as C for element-wise multiplication
    w = np.resize(w, C.shape)  # Resize w to match the shape of C
    Pro_elem = w * C
    menos1 = Pro_elem - 1
    RTA = np.min(Ca * (menos1))
    return RTA


def func_de_retorno_max(C, w, Ca):
    # Ensure w has the same length as C for element-wise multiplication
    w = np.resize(w, C.shape)  # Resize w to match the shape of C
    Pro_elem = w * C
    menos1 = Pro_elem - 1
    RTA = Ca * np.max(menos1)
    return RTA


class Robot:
    def __init__(self, f, Id, T):
        self.Id = Id
        self.f = f
        self.h = np.zeros(T)
        for i in range(T):
            self.h[i] = np.random.uniform(1.0, 100.0)
        self.h /= np.sum(self.h)
        # Initialize r with a random value or a specific value
        # r should have the same length as C for element-wise multiplication in func_de_retorno
        self.r = np.random.rand(len(C))  # Initialize with random numbers between 0 and 1
        self.rate = 0.05  # Initialize mutation rate

    # The following methods should be indented to be part of the Robot class
    def Mutate(self):
        self.r += np.random.normal(loc=0., scale=self.rate, size=len(self.r))

    def GetR(self):
        return self.r

    def SetFitness(self):
        self.Fitness = self.f(C, self.GetR(), Ca)  # Pass 'Ca' to 'f'

        # Fine tunning
        if self.Fitness > 0.05:
            self.rate = 0.005

    def GetFitness(self):
        return self.Fitness


def GetRobots(N, tam):
    Robots = []

    for i in range(N):
        r = Robot(func_de_retorno, i, tam)
        Robots.append(r)

    return Robots


Robots = np.array(GetRobots(500, len(C)))


def GetRobots(N, tam):
    Robots = []

    for i in range(N):
        r = Robot(func_de_retorno, i, tam)
        Robots.append(r)

    return Robots


Robots = np.array(GetRobots(50, len(C)))

for i in range(len(Robots)):
    print(Robots[i].h)


# tasa de exito

def Genetic(Robots, epochs = 500):


    # Guardamos los mas aptos
    N = int(0.5*len(Robots))

    # Mirar la aptitud más pequeña en cada epoca
    Fitness = np.array([])

    for e in range(int(epochs)):

        # actualizamos informaicíón
        for i,p in enumerate(Robots):
            p.Mutate()
            p.SetFitness()

        #clear_output(wait=True)


        # Ordenamos los robots por fitness
        scores = [ (p.Fitness, p) for p in Robots ]
        scores.sort(key = lambda x: x[0], reverse = False)

        # Tomamos lo incides hasta el % porciento
        Temp = [r[1] for i,r in enumerate(scores) if i < N ]

        for i,r in enumerate(Robots):
            j = i%N
            Robots[i] = copy.deepcopy(Temp[j])


        # Miramos el fitness de cada robot
        Fitness_ = [  r.GetFitness() for i,r in enumerate(Robots) ]

        # Guardamos la aptitud promedio
        Fitness = np.append(Fitness,np.mean(Fitness_))

        if Fitness[-1] < 0.0001:
            print('Entrenamiento terminado')
            break

        #ax.legend(loc='upper center', ncol=1, bbox_to_anchor=(-0.2,0.9),frameon=False,fontsize=10)
        if e % 10 == 0:
            clear_output(wait=True)
            # Imagen para pintar
            ax,ax1 = Plotter(e)
            for i,p in enumerate(Robots):
                ax.scatter(p.GetR()[0],p.GetR()[1],marker='.',color='r',label='Id {}'.format(p.Id))
            ax.set_xlim(-5,5)
            ax.set_ylim(-5,5)

            # Funcion de Fitness
            ax1.set_title('Fitness function')
            ax1.plot(Fitness,color='k',label='Id {}, {:.2f}, {:.2f}, {:.2f}'.format(Robots[0].Id, Robots[0].GetR()[0],Robots[0].GetR()[1],Robots[0].GetR()[2]))
            ax1.set_xlabel('Epoch')
            ax1.legend(loc=1)

            plt.show()
Genetic(Robots)

params = np.array([ p.GetR() for i, p in enumerate(Robots)]).mean(axis=0)
params

# punto g

Pesos = np.array([0.185, 0.152, 0.137, 0.125, 0.116, 0.107, 0.096, 0.082])

# ganacia minima
min_ganancia = func_de_retorno(C, Pesos, Ca)
print("En el peor de los casos el retorno seriade",min_ganancia)
#pocible maximo del retorno de inverción
max_ganancia=func_de_retorno_max(C,Pesos,Ca)
print("el retorno maximo posible seria de",max_ganancia)

#cuanto se invierte en cada opcion
invercion=np.zeros(len(Pesos))
for i in range(len(Pesos)):
  invercion[i]=Pesos[i]*Ca
print(invercion)
Total=np.sum(invercion)
print(Total)




[0.21702095 0.00321659 0.16327776 0.1783217  0.08076364 0.00601537
 0.13508562 0.21629837]
[0.12803023 0.05588193 0.18684623 0.01634281 0.19010802 0.09783688
 0.17745549 0.14749842]
[0.23577967 0.15578992 0.10202172 0.1013747  0.22085903 0.07656109
 0.04726675 0.06034711]
[0.13060554 0.05894474 0.19415964 0.13549573 0.21290251 0.16275924
 0.0772889  0.0278437 ]
[0.11969121 0.22297915 0.07826155 0.02339778 0.06604502 0.14784548
 0.11604018 0.22573964]
[0.07851888 0.14509701 0.05684064 0.0112486  0.15155467 0.17200482
 0.19156536 0.19317003]
[0.02139629 0.02605764 0.23297587 0.18009095 0.17565767 0.17008652
 0.01018184 0.1835532 ]
[0.01339952 0.07292156 0.21902775 0.06951528 0.0432026  0.10368794
 0.23325877 0.24498659]
[0.18325027 0.17893512 0.04611691 0.14109119 0.07073689 0.00450792
 0.15830547 0.21705623]
[0.04577379 0.15324083 0.16059889 0.08161838 0.15713923 0.1086455
 0.12646304 0.16652033]
[0.12137811 0.13236752 0.05796471 0.18984588 0.15983231 0.10179977
 0.17653362 0.06027807]


No seria rentable realizar una ivercion de esta forma debido al riesgo de perdida frente a la baja posibilidad de ganancia siendo ma probable perder dinero que ganarlo sinedo que para el mejor de los casos el retorno es de 1024580 lo que da como margen de ganacia neto tan solo 24580 pesos.